# QA ODP files

expeditions 101 - 210; range_tables for paleo fossils, age_models; https://www.ngdc.noaa.gov/mgg/geology/data/joides_resolution/odp_all_paleontology.tar.gz

In [1]:
import sys
import os 

sys.path.append('../scripts/')
import glob


import pandas as pd
import chardet

## QA ODP paleo files

count the number of good files vs files that need to be fixed

In [2]:
data_path = 'cleaned_data/odp_all_paleontology/range_tables'
raw_csvs = glob.glob(f"{data_path}/**/*.txt", recursive=True)

In [3]:
len(raw_csvs)

2045

In [4]:
bad_encoding = [] # does not use utf-8 encoding
bad_tabs = [] # rows have different number of columns than headers
space_delim = [] # uses spaces as delimiter
missing_fields = [] # does not have all the fields
good_files = []

expected_fields = {
    'Age From (oldest)', 'Age To (youngest)', 'Zone From (bottom)', 'Zone To  (top)',
    'Leg', 'Site','H', 'Cor', 'T', 'Sc', 'Top(cm)', 'Depth (mbsf)',  
    'Group Abundance', 'Group Preservation'
}


for file in raw_csvs:
    try:
        df = pd.read_csv(file, delimiter="\t")
    except UnicodeDecodeError:
        bad_encoding.append(file)
        continue
    except pd.errors.ParserError:

        bad_tabs.append(file)
        continue
    
    
    if expected_fields.issubset(df.columns):
        good_files.append(file)
    elif len(df.columns) == 1:
        space_delim.append(file)
    else:
        missing_fields.append(file)

In [5]:
print('bad_tabs', len(bad_tabs))
print('bad_encoding', len(bad_encoding))
print('space_delim', len(space_delim))
print('missing_fields', len(missing_fields))
print('good_files', len(good_files))

bad_tabs 0
bad_encoding 0
space_delim 0
missing_fields 0
good_files 2045


## process latin_encoding
handle files with encoding that isn't utf-8

In [6]:
for file in bad_encoding:
    print(file)

convert file to utf-8 encoding
https://codereview.stackexchange.com/a/202985

In [7]:
for file in bad_encoding:
    with open(file, 'rb') as f:
        content_bytes = f.read()
    detected = chardet.detect(content_bytes)
    encoding = detected['encoding']
    content_text = content_bytes.decode(encoding)
    
    with open(file, 'w', encoding='utf-8') as f:
        f.write(content_text)
        print(file)

## process bad_tabs
handle files where the hearers and rows have different number of columns

In [8]:
for file in bad_tabs:
    print(file)

## process space_delim

handle files that use random number of spaces to separate the columns

In [9]:
for file in space_delim:
    file_size = os.path.getsize(file)
    print(f'{file}: {file_size}')

https://stackoverflow.com/questions/19759423/convert-a-space-delimited-file-to-comma-separated-values-file-in-python


## process missing_fields

handle files don't have the 

In [10]:
for file in missing_fields:
    all_fields = set()
    df = pd.read_csv(file, delimiter="\t")
    file_fields = list(df.columns)[1:17]
    
    all_fields.update(expected_fields)
    all_fields.update(file_fields)
    print(all_fields)